In [14]:
import pandas as pd
# import numpy as np
# import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import helper_funtions


In [15]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

In [16]:
PATH = helper_funtions.get_path()

# Read data

In [17]:
df = pd.read_csv(PATH + 'data/data_cleaning.csv')
df.sample(n=5).head()

,UUID_client,Age,Location,Income,TAX,Previous_sales,Type_products,Contact_channel,Contact_hour,Num_contacts,Satisfaction_score,Sales
51,fc13841d-8ea7-5f38-a819-5ae8e6605a41,22,IL,80677.0,12101.55,1,B,Online Chat,11:30:00,1,3,0
604,b9a48147-8d91-5b40-ad1d-0000ef84ad44,46,IL,66751.0,10012.65,3,C,Email,18:00:00,1,1,0
802,4ffd603e-3a3d-53ea-9c87-00cb35321b63,39,CA,59118.0,8867.70,0,B,Online Chat,06:45:00,1,1,0
896,58744027-984c-59ae-a106-248517a30de3,37,CA,43230.0,6484.50,2,C,Phone,10:45:00,4,5,1
879,69e4c167-f9c8-5080-b012-87fd21b90ed5,44,TX,60090.0,9013.50,2,B,Online Chat,17:00:00,3,3,1


# EDA

## Describe data

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   UUID_client         1000 non-null   object 
 1   Age                 1000 non-null   int64  
 2   Location            1000 non-null   object 
 3   Income              996 non-null    float64
 4   TAX                 996 non-null    float64
 5   Previous_sales      1000 non-null   int64  
 6   Type_products       1000 non-null   object 
 7   Contact_channel     1000 non-null   object 
 8   Contact_hour        1000 non-null   object 
 9   Num_contacts        1000 non-null   int64  
 10  Satisfaction_score  1000 non-null   int64  
 11  Sales               1000 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [19]:
df.describe()

,Age,Income,TAX,Previous_sales,Num_contacts,Satisfaction_score,Sales
count,1000.000000,996.000000,996.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,41.191000,63100.329317,9465.049398,2.052000,3.492000,2.558000,0.269000
std,12.259234,21638.692537,3245.803881,2.635952,2.399102,1.331318,0.443662
min,18.000000,5000.000000,750.000000,-7.000000,1.000000,1.000000,0.000000
25%,31.000000,46277.750000,6941.662500,0.000000,2.000000,1.000000,0.000000
50%,40.500000,62770.500000,9415.575000,1.000000,3.000000,2.000000,0.000000
75%,51.000000,80618.000000,12092.700000,4.000000,5.000000,3.000000,1.000000
max,125.000000,165355.000000,24803.250000,25.000000,33.000000,5.000000,1.000000


In [20]:
print('Suma de valores faltantes en el conjunto de datos:')
df.isnull().sum()

Suma de valores faltantes en el conjunto de datos:


UUID_client           0
Age                   0
Location              0
Income                4
TAX                   4
Previous_sales        0
Type_products         0
Contact_channel       0
Contact_hour          0
Num_contacts          0
Satisfaction_score    0
Sales                 0
dtype: int64

Observaciones:
1. La cantidad de compras que el cliente ha realizado en el pasado cuenta con valores negativos lo que no corresponde con un registro lógico.
2. Existen usuarios con edades mayores a 70 años que parecen ser datos incorrectos
3. Existen 181 usuarios con interacciones superiores a cinco, de las cuales, una alcanza las 33 interacciones.
4. La mitad de la muestra califica la satisfacción entre un nivel uno y dos (muy bajo).
5. Al menos la mitad de los clientes no tuvieron una venta efectiva.

## Visualization

### Categorical data

In [21]:
categorical_columns = [
    'Location', 'Type_products', 'Contact_channel',
    'Satisfaction_score', 'Sales'
]
fig = make_subplots(rows=5, cols=1, subplot_titles=categorical_columns)
row, col = 1, 1
for column in categorical_columns:
    helper_funtions.dataviz_structure_categorical(column, fig, df, row, col)
    row += 1
fig.update_layout(height=800, width=750, title_text='Variables categóricas')
fig.show()

Observaciones:
1. La distribución de ubicaciones es homogenea.
2. No parece existir un tipo de producto preferido por los clientes.
3. El canal de preferencia de contacto del cliente es el teléfono.
4. Los niveles de satisfacción son bajos a nivel de muestra.
5. la efectividad de ventas es baja (variable desvalanceada).

### Numerical data

In [22]:
numerical_columns_histogram = [
    'Previous_sales', 'Num_contacts'
]

fig = go.Figure()
for column in numerical_columns_histogram:
    fig.add_trace(go.Histogram(x=df[column], name=column))
fig.update_layout(barmode='stack')
fig.show()

In [23]:
# Determine peak times by rounding the time in groups of one hour 
df['Contact_hour'] = pd.to_datetime(df['Contact_hour'], format='%H:%M:%S')
df['Contact_hour_round'] = df['Contact_hour'].apply(lambda x: x.round('60 min'))

# fig = px.histogram(df, x='Contact_hour_round')
# fig.show()

# df_peak_time = df['Contact_hour_round'].value_counts().head()
# df_peak_time = df_peak_time.rename_axis('Hour')
# df_peak_time = df_peak_time.reset_index(name='Count')
# peak_hours = []
# for i in range(len(df_peak_time)):
#     peak_hours.append(df_peak_time['Hour'].iloc[i])

# for i in range(len(df)):
#     if df.loc[i, 'Contact_hour_round'] in peak_hours:
#         df.at[i, 'Is_peak_hour'] = 1
#     else:
#         df.at[i, 'Is_peak_hour'] = 0

# del df_peak_time

numerical_columns_histogram_hour = [
    'Contact_hour', 'Contact_hour_round'
]

fig = go.Figure()
for column in numerical_columns_histogram_hour:
    fig.add_trace(go.Histogram(x=df[column], name=column))
fig.update_layout(barmode='stack')
fig.show()

In [24]:
numerical_columns_boxplot = [
    'Age', 'Income', 'TAX', 'Previous_sales'
    ]

fig = make_subplots(rows=4, cols=1, subplot_titles=numerical_columns_boxplot)
row, col = 1, 1
for i in range(len(numerical_columns_boxplot)):
    column = numerical_columns_boxplot[i]
    helper_funtions.dataviz_structure_numerical(column, fig, df, row, col)
    row += 1
fig.update_layout(height=2000, width=500, title_text='Variables Numéricas')
fig.show()

Observaciones:
1. Pese a la existencia de compras negativas en el conjunto de datos (a ajustar en el procesamiento) se evidencia una asimetría positiva.
2. El número de intentos de contacto realizados por el cliente cuenta con una asimetría positiva.
3. Se evidencian puntos pico a determinadas horas del día.
4. Existen valores atípicos que deben ser considerados durante la etapa de procesamiento de datos.

# Univariate analysis

In [25]:
helper_funtions.conversion_rate_chart('Age', 'Sales', df)

df.loc[:,'groups_ages'] = '18-20'
df.loc[df['Age']>60,'groups_ages'] = '>60'
df.loc[(df['Age']>40)&(df['Age']<=60), 'groups_ages'] = '41-60'
df.loc[(df['Age']>17)&(df['Age']<=40), 'groups_ages'] = '20-40'

# Y grafiquemos la tasa de conversión para esta nueva columna
helper_funtions.conversion_rate_chart('groups_ages', 'Sales', df, type='bar')

In [26]:
helper_funtions.conversion_rate_chart('Income', 'Sales', df, type='scatter')